# Puviyan Soil Detection v6 - Flutter Compatible

**Repository**: https://github.com/ctopuviyan/puviyan-ai-training

**Compatibility**: Flutter tflite_flutter ^0.11.0

**Features**:
- ✅ Flutter TensorFlow Lite compatibility
- ✅ FULLY_CONNECTED operations ≤ version 11
- ✅ TFLITE_BUILTINS operations only
- ✅ Mobile-optimized architecture
- ✅ Validation system integration

---

In [ ]:
# Setup and Imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import cv2
import os
import json
import zipfile
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

print("🚀 Puviyan Soil Detection v6 - Flutter Compatible")
print("=" * 60)
print(f"📱 Target: Flutter tflite_flutter ^0.11.0")
print(f"🔧 TensorFlow version: {tf.__version__}")
print(f"🎯 Compatibility: FULLY_CONNECTED ≤ v11, TFLITE_BUILTINS only")

In [ ]:
# Flutter Compatibility Configuration
class FlutterCompatibilityConfig:
    """Configuration ensuring Flutter TensorFlow Lite compatibility"""
    
    FLUTTER_TFLITE_VERSION = "0.11.0"
    MAX_FULLY_CONNECTED_VERSION = 11
    INPUT_SIZE = 224
    NUM_CLASSES = 8
    BATCH_SIZE = 32
    
    SOIL_CLASSES = [
        "Alluvial Soil", "Black Soil", "Red Soil", "Laterite Soil",
        "Desert Soil", "Saline/Alkaline Soil", "Peaty/Marshy Soil", "Forest/Hill Soil"
    ]
    
    @staticmethod
    def get_flutter_compatible_converter(model):
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter._experimental_lower_tensor_list_ops = False
        return converter

print("✅ Flutter compatibility configuration loaded")

In [ ]:
# Data Setup
def setup_data_directories():
    os.makedirs('soil_data/train', exist_ok=True)
    os.makedirs('soil_data/validation', exist_ok=True)
    os.makedirs('soil_data/test', exist_ok=True)
    os.makedirs('models', exist_ok=True)
    print("📁 Data directories created")
    print("📝 Upload your soil dataset to soil_data/train/[class_name]/")

setup_data_directories()

In [ ]:
# Flutter-Compatible Model Architecture
def create_flutter_compatible_soil_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(224, 224, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.GlobalAveragePooling2D(),  # Mobile-optimized
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(8, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', 'top_3_accuracy']
    )
    
    print(f"✅ Flutter-compatible model created ({model.count_params():,} parameters)")
    return model

model = create_flutter_compatible_soil_model()
model.summary()

In [ ]:
# Create Data Generators
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=30, width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, vertical_flip=True, brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(rescale=1./255)

try:
    train_generator = train_datagen.flow_from_directory(
        'soil_data/train', target_size=(224, 224), batch_size=32,
        class_mode='categorical', classes=FlutterCompatibilityConfig.SOIL_CLASSES
    )
    
    val_generator = val_datagen.flow_from_directory(
        'soil_data/validation', target_size=(224, 224), batch_size=32,
        class_mode='categorical', classes=FlutterCompatibilityConfig.SOIL_CLASSES
    )
    
    print(f"✅ Data generators created: {train_generator.samples} train, {val_generator.samples} val")
except:
    print("⚠️ Upload your dataset first, then run this cell")

In [ ]:
# Training Configuration and Execution
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=8, min_lr=1e-7),
    tf.keras.callbacks.ModelCheckpoint('models/best_puviyan_soil_v6.h5', monitor='val_accuracy', save_best_only=True)
]

# Train the model
print("🎯 Starting training...")
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=1
)

print("✅ Training completed!")

In [ ]:
# Convert to Flutter-Compatible TensorFlow Lite
def convert_to_flutter_tflite(model, output_path='puviyan_soil_flutter_v6.tflite'):
    print("🔄 Converting to Flutter-compatible TFLite...")
    
    try:
        converter = FlutterCompatibilityConfig.get_flutter_compatible_converter(model)
        
        # Representative dataset for quantization
        def representative_dataset():
            for _ in range(100):
                data = np.random.random((1, 224, 224, 3)).astype(np.float32)
                yield [data]
        
        converter.representative_dataset = representative_dataset
        tflite_model = converter.convert()
        
        with open(output_path, 'wb') as f:
            f.write(tflite_model)
        
        size_mb = len(tflite_model) / 1024 / 1024
        print(f"✅ Flutter-compatible model saved: {output_path} ({size_mb:.2f} MB)")
        
        # Verify compatibility
        interpreter = tf.lite.Interpreter(model_path=output_path)
        interpreter.allocate_tensors()
        print("✅ Flutter compatibility verified!")
        
        return output_path
    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        return None

tflite_path = convert_to_flutter_tflite(model)

In [ ]:
# Create Flutter Labels and Metadata
labels_data = {
    "labels": FlutterCompatibilityConfig.SOIL_CLASSES,
    "model_info": {
        "version": "6.0.0",
        "compatibility": f"flutter_tflite_{FlutterCompatibilityConfig.FLUTTER_TFLITE_VERSION}",
        "input_size": FlutterCompatibilityConfig.INPUT_SIZE,
        "num_classes": FlutterCompatibilityConfig.NUM_CLASSES,
        "created_date": datetime.now().isoformat(),
        "repository": "https://github.com/ctopuviyan/puviyan-ai-training"
    },
    "preprocessing": {
        "input_format": "RGB",
        "normalization": "0_to_1",
        "resize_method": "bilinear"
    }
}

with open('labels_flutter_v6.json', 'w') as f:
    json.dump(labels_data, f, indent=2)

print("✅ Flutter labels file created: labels_flutter_v6.json")

In [ ]:
# Create Flutter Package
zip_path = 'puviyan_flutter_models_v6.zip'

with zipfile.ZipFile(zip_path, 'w') as zipf:
    if os.path.exists('puviyan_soil_flutter_v6.tflite'):
        zipf.write('puviyan_soil_flutter_v6.tflite')
        print("📱 Added TFLite model")
    
    if os.path.exists('labels_flutter_v6.json'):
        zipf.write('labels_flutter_v6.json')
        print("🏷️ Added labels file")

print(f"✅ Flutter package created: {zip_path}")
print("\n📋 Flutter Integration:")
print("1. Download the zip file")
print("2. Extract to Flutter project assets/models/")
print("3. Update model paths in Flutter code")
print("4. Test with validation system")
print("\n🎉 Ready for Flutter deployment!")